# Relativistic Raytracer

or, a guide in three days of procrastination and a 30% grade deduction

*brought to you by @chas-card, @owl10124, and @TheRealOrange*

## Motivation

I don't think the motivation for a relativistic raytracer needs to be explained. It's a relativistic raytracer.

In [1]:
from array import array
from PIL import Image
import numpy as np
import math

np_type = np.float32
c = 8 #3e8

FARAWAY = 1.0e+39  # A large distance

def norm(arr): return arr/np.sqrt(np.sum(np.square(arr),axis=0))
def lt_velo(lt, velo):
    if len(np.shape(velo))==1: velo=velo[:,np.newaxis]
    v4 = np.concatenate((np.array([c*np.ones(np.shape(velo)[1])]), velo), axis=0)
    vp4 = lt @ v4
    return vp4[1:]*c/vp4[0]

## Frame

The obvious important concept in SR: the transform between inertial reference frames.

Each frame has a velocity vector, giving rise to its own LT.

We also define these relative to another frame, allowing us to stack the transformations. At the 'bottom' is a world frame.

In [2]:
# frame class to handle defining different reference frames with respect to other reference frames
# world frame is just (0, 0, 0, 0) position, use it as a "special" frame to transform between any frames
class Frame:
    def __init__(self, velocity, ref=None):
        self.velocity = np.array(velocity, dtype=np_type)
        self.ref = ref

    @property
    def lt(self):
        b=self.velocity/c
        b2 = np.sum(np.square(b))
        assert (b2 <= 1)

        g = 1 / (np.sqrt(1 - b2))

        lt_mat = np.eye(4, dtype=np_type)
        if b2==0: return lt_mat

        lt_mat[0, 0] = g
        lt_mat[0, 1:] = lt_mat[1:, 0] = -b * g
        lt_mat[1:, 1:] += (g - 1) * np.matmul(b[np.newaxis].T, b[np.newaxis]) / b2

        assert(abs(np.linalg.det(lt_mat)-1)<1e-3)
        return lt_mat

    @property
    def inv_lt(self):
        inv_lt_mat = self.lt
        inv_lt_mat[0, 1:] = -inv_lt_mat[0, 1:]
        inv_lt_mat[1:, 0] = -inv_lt_mat[1:, 0]
        return inv_lt_mat

    @property
    def to_world_lt(self):
        if self.ref is None:
            return self.inv_lt
        else:
            return np.matmul(self.ref.to_world_lt, self.inv_lt)

    @property
    def from_world_lt(self):
        if self.ref is None:
            return self.lt
        else:
            return np.matmul(self.lt, self.ref.from_world_lt)

    def compute_lt_to_frame(self, frame):
        return np.matmul(frame.from_world_lt, self.to_world_lt)

    def compute_lt_from_frame(self, frame):
        return np.matmul(self.from_world_lt, frame.to_world_lt)

    def __str__(self):
        return "Frame with velocity "+str(self.velocity)+" wrt "+str(self.ref)

## Camera

The screen / camera class does exactly what one expects.